In [ ]:
import json
import os
from os.path import realpath, dirname, join

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

ROOT = './data/'
OUTPUT_DIR = join(ROOT, 'plots')
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
sns.set_style("whitegrid")
sns.set_palette("PuBuGn_d")

In [ ]:
def extract_values(model_name, row):
    extracted = pd.DataFrame(columns=['Model', 'Benchmark', 'Score'])
    s = pd.Series(index=extracted.columns)
    for k, entry in row.items():
        s['Model'] = model_name
        
        if isinstance(entry, str):
            l = json.loads(entry.replace('(', '[').replace(')', ']'))
            # Pairs of numbers represent (spearman, pearson)
            s['Benchmark'] = k + '\nspearman'
            s['Score'] = float(l[0])
            extracted = extracted.append(s, ignore_index=True)
            #s['Benchmark'] = k + ' - pearson'
            #s['Score'] = float(l[1])
            #extracted = extracted.append(s, ignore_index=True)
        else:
            s['Benchmark'] = k
            s['Score'] = entry
            extracted = extracted.append(s, ignore_index=True)
    return extracted

def extract_model_name(row):
    import json
    n = row['outputmodelname']
    try:
        s = json.loads(n.replace("'", '"'))
        return s[0]
    except ValueError as e:
        print(e)
        pass
    return n

def plot_evaluation_results(eval_filenames):
    global extracted
    extracted = None
    for fname in eval_filenames:
        df = pd.read_csv(fname, sep=';', header=0)
        for i, row in df.iterrows():
            evals = json.loads(row['downstream_tasks'].replace("'", '"'))
            ee = extract_values(extract_model_name(row), row[evals])
            if extracted is None:
                extracted = ee
            else:
                extracted = extracted.append(ee)
          
    fig, ax = plt.subplots(2, 1, figsize=(16, 2*7))
    for i, coeff_metrics in enumerate([False, True]):
        mask = extracted['Benchmark'].str.contains("spearman|pearson")
        if not coeff_metrics:
            mask = ~mask
        selected = extracted[mask]
        
        sns.barplot(x='Benchmark', y='Score', hue='Model', data=selected, ax=ax[i])
        ax[i].set_title('Evaluation results on several variants')
        ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=45, horizontalalignment='right')
        
    plt.tight_layout()
    plt.savefig(join(OUTPUT_DIR, 'comparison.pdf'), bbox_inches='tight', dpi=128)

In [ ]:
sources = [
    'evaluation-02/hybrid.csv',
    'evaluation-cnmow-01/cnmow.csv',
]
plot_evaluation_results([join(ROOT, source) for source in sources])